In [1]:
import sys, os, time
# sys.path.append('/Users/calum/Developer/pyzx-heuristics-master')
# import pyzx as zx_heuristics
sys.path.append('../..')
import pyzx as zx
from benchmarking import benchmark

In [2]:
b = benchmark(dirpath='benchmark')
# b = benchmark()
# b.load_circuits(os.path.join('..', '..', 'circuits', 'benchmarking', 'Fast', 'before'), group_name='fast')
# b.load_circuits(os.path.join('..', '..', 'circuits', 'benchmarking', 'Fast', 'nrscm'), group_name='fast', simp_strategy='NRSCM')
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
Loaded functions:  ['basic', 'full', 'flow']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,NRSCM,basic,flow,full
fast,Y,Y,Y,Y,Y


In [3]:
def basic_optimize(c):
    c1 = zx.optimize.basic_optimization(c.copy(), do_swaps=False).to_basic_gates()
    c2 = zx.optimize.basic_optimization(c.copy(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2
    return c1

def full_reduce(c):
    g = c.to_graph()
    zx.simplify.full_reduce(g,quiet=True)
    c2 = zx.extract_circuit(g,up_to_perm=False).to_basic_gates()
    return basic_optimize(c2)

def flow_optimize(c):
    g = c.to_graph()
    zx.simplify.teleport_reduce(g, store=False)
    zx.simplify.to_graph_like(g)
    zx.simplify.flow_2Q_simp(g)
    c2 = zx.extract_simple(g,up_to_perm=False).to_basic_gates()
    return basic_optimize(c2)

In [4]:
b.add_simplification_func(basic_optimize, 'basic', groups_to_run=['fast'],verify=True)
b.add_simplification_func(full_reduce, 'full', groups_to_run=['fast'],verify=True)
b.add_simplification_func(flow_optimize, 'flow2', groups_to_run=['fast'],verify=True)

100%|██████████| 32/32 [00:00<00:00, 729444.17it/s]
Processing flow2 on qft_4                                             : 100%|██████████| 32/32 [02:31<00:00,  4.73s/it]


In [5]:
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
Loaded functions:  ['basic', 'full', 'flow', 'flow2']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,NRSCM,basic,flow,flow2,full
fast,Y,Y,Y,Y,Y,Y


In [6]:
df = b.df(groups=['fast'],
     routines='all',
     funcs='all',
     atts=['Qubits', '2Q Count', 'T Count'])

In [8]:
# funcs = ['basic','full_reduce']
# t_probs = [0.015*i for i in range(11)]
# fig = b.Pt_graphs(funcs=funcs, qubits=8, depth=400, cnot_prob=0.3, t_probs=t_probs, ys=['Gates','2Q Count','T Count'], reps=20, overwrite=False, random_seed=42)

In [7]:
b.save(dirpath='benchmark')